In [1]:
import random
from math import inf


params = {
    "farmer_productivity": 1.5,
    "land_capacity": 120,

    "steal_rate": 3,
    "suppression_prob": 0.3,
    "confiscate_rate": 0.7,

    "tax_rate": 1.5,
    "soldier_base_pay": 2.0,

    "alpha_perception": 0.25,
    "fear_increment": 0.1,
    "fear_decay": 0.2,

    "utility_margin": 0.5,
    "switch_urge_threshold": 2.0,
}

EPS = 1e-6
FEAR_MAX = 10.0


class Person:
    """
    Represents a single agent in the simulation.

    Each person has:
    - An economic role: farmer, bandit, or soldier
    - Economic state: wealth and income flows
    - Psychological state: fear, perceived violence, loyalty
    - Behavioral state: role-switching logic with inertia + cooldown
    - Individual heterogeneity: productivity, aggression, and enforcement skill

    This class handles:
    - Individual actions (produce, rob, suppress, pay tax, receive pay)
    - Psychological updates
    - Forward-looking utility prediction
    - Role switching decisions
    """

    def __init__(self, role, wealth=0.0, prod_mult=1.0, aggr_mult=1.0, skill_mult=1.0):
        """
        Initialize a new agent.

        Parameters:
        - role: Initial role ("farmer", "bandit", or "soldier")
        - wealth: Initial wealth level
        - prod_mult: Productivity multiplier (heterogeneity)
        - aggr_mult: Aggression multiplier (how much they steal)
        - skill_mult: Skill multiplier for suppression
        """
        self.role = role
        self.wealth = max(0.0, wealth)

        # ---心理 / psychological traits ---
        self.fear = 0.0                         # internal fear level
        self.perceived_violence = 0.0           # smoothed estimate of world violence
        self.loyalty = random.uniform(0.8, 1.2) # resistance to switching roles

        # --- Role switching state ---
        self.next_role = role
        self.age_in_role = 0                   # how long they've stayed
        self.switch_cooldown = 3               # inertia to prevent constant switching

        # --- Last tick economics (for logging/diagnostics) ---
        self.last_production = 0.0
        self.last_rob_gain = 0.0
        self.last_pay = 0.0

        # --- Heterogeneity among agents ---
        # These override the passed base values with randomness
        self.prod_mult = random.uniform(0.9, 1.1)
        self.aggr_mult = random.uniform(0.9, 1.1)
        self.skill_mult = random.uniform(0.9, 1.1)

    # ================= Housekeeping ================= #

    def reset_for_new_tick(self):
        """
        Reset per-tick variables and update internal counters.
        Called once at the beginning of every world step.
        """
        self.next_role = self.role
        self.age_in_role += 1

        # cooldown prevents constant oscillating behavior
        if self.switch_cooldown > 0:
            self.switch_cooldown -= 1

        # reset logs
        self.last_production = 0.0
        self.last_rob_gain = 0.0
        self.last_pay = 0.0

    # ================= Role detectors ================= #

    def is_farmer(self):
        """Return True if this agent is a farmer."""
        return self.role == "farmer"

    def is_bandit(self):
        """Return True if this agent is a bandit."""
        return self.role == "bandit"

    def is_soldier(self):
        """Return True if this agent is a soldier."""
        return self.role == "soldier"

    # ================= Role actions ================= #

    def produce(self, farmer_productivity, scale=1.0):
        """
        Farmer produces wealth.

        Production depends on:
        - Fixed productivity parameter
        - Individual productivity multiplier
        - Optional global scale (for resource scarcity, etc.)
        """
        if not self.is_farmer():
            return 0.0

        inc = max(0.0, self.prod_mult * farmer_productivity * max(0.0, scale))
        self.wealth += inc
        self.last_production = inc
        return inc

    def attempt_rob(self, victim, steal_rate, fear_bump_robbed=0.3):
        """
        Bandit attempts to steal from a farmer.

        If successful:
        - Transfers stolen wealth
        - Increases victim fear
        - Logs stolen wealth
        """
        if not (self.is_bandit() and victim and victim.role == "farmer"):
            return 0.0, False

        stolen = min(self.aggr_mult * steal_rate, victim.wealth)

        if stolen <= 0:
            return 0.0, False

        victim.wealth -= stolen
        victim.fear = min(FEAR_MAX, victim.fear + fear_bump_robbed)

        self.wealth += stolen
        self.last_rob_gain = stolen

        return stolen, True

    def attempt_suppress(self, target_bandit, p_hit, confiscate_rate, fear_bump_suppressed=0.2):
        """
        Soldier attempts to suppress a bandit.

        If successful:
        - Confiscates stolen wealth
        - Raises bandit's fear
        - Gains confiscated wealth
        """

        if not (self.is_soldier() and target_bandit and target_bandit.role == "bandit"):
            return 0.0, False

        hit_prob = min(1.0, max(0.0, self.skill_mult * p_hit))

        if random.random() >= hit_prob:
            return 0.0, False

        amt = min(self.skill_mult * confiscate_rate, target_bandit.wealth)

        target_bandit.wealth -= amt
        target_bandit.fear = min(FEAR_MAX, target_bandit.fear + fear_bump_suppressed)

        self.wealth += amt
        return amt, True

    def pay_tax(self, tax_rate):
        """
        Farmer pays tax as a percentage of wealth.
        Funds are redistributed to soldiers.
        """
        if not self.is_farmer():
            return 0.0

        tax = tax_rate * self.wealth
        self.wealth -= tax

        return tax

    def receive_pay(self, amount):
        """
        Soldier receives salary from tax pool + government base pay.
        """
        if amount > 0:
            self.wealth += amount
            self.last_pay = amount

    # ================= Psychological updates ================= #

    def update_psychology(self, global_violence, alpha, fear_inc, fear_dec):
        """
        Update internal psychological state:
        - Update perceived violence using exponential smoothing
        - Translate perceived violence into fear changes
        """
        a = max(0.0, min(1.0, alpha))

        self.perceived_violence = (1 - a) * self.perceived_violence + a * global_violence

        if self.perceived_violence > 0.5:
            self.fear = min(FEAR_MAX, self.fear + fear_inc)
        else:
            self.fear = max(0.0, self.fear - fear_dec)

    # ================= Forward-looking utility ================= #

    def predicted_utility(self, role, world, params):
        """
        Predict utility of choosing a given role NEXT tick.

        This is a forward-looking expectation based on:
        - Population composition
        - Global violence
        - Tax rates, productivity, and suppression parameters
        """

        # Hypothetical population counts if this person switched
        f, b, s = world.farmer_pop, world.bandit_pop, world.soldier_pop

        if self.is_farmer(): f -= 1
        elif self.is_bandit(): b -= 1
        elif self.is_soldier(): s -= 1

        if role == "farmer": f += 1
        elif role == "bandit": b += 1
        elif role == "soldier": s += 1

        violence = world.total_violence_last_tick

        # --- Farmer utility ---
        if role == "farmer":
            predicted_prod = self.prod_mult * params["farmer_productivity"]
            safety_bonus = max(0.0, 0.5 - 0.02 * violence)
            return predicted_prod + safety_bonus - 0.01 * b

        # --- Bandit utility ---
        if role == "bandit":
            base_rob = self.aggr_mult * params["steal_rate"] if f > 0 else 0.0
            return (
                base_rob
                - 0.01 * b       # bandit competition
                + 0.05 * f       # more farmers = more loot
                - 0.02 * s       # soldiers increase risk
            )

        # --- Soldier utility ---
        if role == "soldier":
            tax_pool = f * params["farmer_productivity"] * params["tax_rate"] if f > 0 else 0.0
            base_total = params["soldier_base_pay"] * s
            pay_per = (tax_pool + base_total) / s if s > 0 else params["soldier_base_pay"]

            return pay_per + 0.02 * b - 0.30

    # ================= Switching logic ================= #

    def decide_switch(self, utilities, switch_urge_threshold, utility_margin, global_tick):
        """
        Decide whether to switch roles.

        Switching depends on:
        - Fear (creates switching urge)
        - Loyalty (resistance factor)
        - Utility comparison
        - Cooldown and startup freeze
        """

        if global_tick <= 5:
            self.next_role = self.role
            return

        if self.switch_cooldown > 0:
            self.next_role = self.role
            return

        urge = self.fear / (self.loyalty + EPS)
        current_u = utilities[self.role]

        if urge <= switch_urge_threshold:
            self.next_role = self.role
            return

        # find best role by utility
        best_role = max(utilities, key=lambda r: utilities[r])

        if utilities[best_role] > current_u + utility_margin:
            self.next_role = best_role

    def finalize_role_switch(self):
        """
        Commit the switch decided earlier and reset role age + cooldown.
        """
        if self.next_role != self.role:
            self.role = self.next_role
            self.age_in_role = 0
            self.switch_cooldown = 1

    # ================= Export ================= #

    def to_dict(self):
        """
        Export agent data for logging, visualization, or analysis.
        """
        return {
            "role": self.role,
            "wealth": round(self.wealth, 3),
            "fear": round(self.fear, 3),
            "perceived_violence": round(self.perceived_violence, 3),
            "loyalty": round(self.loyalty, 3),
            "age_in_role": self.age_in_role,
        }
